## Introduction

STM is a text mining technique, initially conceived for the analysis of political texts, which has been extensively adopted in social sciences (<a href="https://www.structuraltopicmodel.com/">here</a> you can find a list of the main publications that have adopted STM). As other topic models, like Latent Dirichlet Allocation it basically allows to identify abstract "topics" that occur in a collection of documents, but compared to other models, it allows the analysis of relationships with document metadata, in form of co-variates, either in terms of the degree of association of a document to a topic, either of the association of a word to a topic. As an example, it is possible to take a bunch of posts published on different political blogs in the months before an election, and see which topics were prevalent in the posts of blogs of a certain political leaning (in this case, political leaning of the blog is used as co-variate for topical prevalence), or to see how words associated to the treatment of a specific topic change depending to the political affiliation (in this case, political leaning of the blog is used as co-variate for topical content) (you can refer to the  <a href="https://cran.r-project.org/web/packages/stm/vignettes/stmVignette.pdf">R package vignette</a> for more details).

As I will stress all over, as the sample is totally arbitrary, any result will not have any statistical validity whatsoever. This is simply meant to be an attempt to explore a technique (and a R package) which can have several potential applications, both in terms of analytical purposes, both in terms of information visualisation (allowing for example to get over the use and abused word cloud). 

As many topics in data analysis, it is something easier to do rather than to explain, and something that can be really understood only when you get your hands dirty with some data, which is what prompted me to try this. In this and the following entries, I will try my hand and post the results of some attempts I am making with STM, more specifically testing the model on some job offers extracted from Indeed UK. As this is a work in progress, I will post the results of my work as I get through them, so I am not really sure where this will lead, but I hope to have some fun along the way. 


## Part I - Web Scraping

In this first post I will not really get my hands on STM yet, but I will illustrate how I obtained the textual data I will use in the rest of the work. As mentioned above, I decided to focus on job offers: there is no specific reason for this, other than that I considered this a good example of texts that offer some metadata to incorporate in the analysis (type of job, salary, location), and whose topicality identification could represent a good test for the model. The choice fell on indeed.co.uk also for no specific reason, and I stuck to it after I noticed that scraping it was relatively easy (although the quality of metadata, as we will see later on, is not the best we could have hoped for). 
Obviously, if I had access to indeed.co.uk API this whole process would have been probably quicker, but since I don’t, I scraping the site was the only feasible option. 
The first step was to create three accessory functions to obtain from each offer page the information needed. This was relatively easily done thanks to htmlParse from XML package:

In [ ]:
#load necessary libraries
suppressWarnings(library(rvest))
suppressWarnings(library("xml2"))
suppressWarnings(library("XML"))
suppressWarnings(library("stringr"))
suppressWarnings(library(dplyr))
suppressWarnings(library(naniar))

## Scrape the info from pages:
#metadata 
getmetadataindeed<-function(url){
  meta<-read_html(url)%>%
    htmlParse( asText=TRUE)%>%xpathSApply( "//*[contains(@class,'jobsearch-JobMetadataHeader-iconLabel')]", xmlValue)
  if (is.list(meta)) {meta<-NA
  } #as not all the job descriptions contain metadata, this was introduced to avoid ending up with empty lists in the dataframe
meta 
  }

#job description
getjobdescindeed<-function(url){
  read_html(url)%>%
    htmlParse( asText=TRUE)%>%xpathSApply( "//*[contains(@id,'jobDescriptionText')]", xmlValue)%>%paste(collapse=', ')%>%str_replace_all("\n", " ")
  
}

#job title 
getjobtitle<-function(url){
  tit<-read_html(url)%>%
    htmlParse( asText=TRUE)%>%xpathSApply( "//*[contains(@class, 'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title')]", xmlValue)
  if (is.list(tit)) {
    tit<-NA
  }
  tit
}

As these functions need to be fed with the specific URLs of the job offer webpages, and do this by hand for hundreds of offers is not really an option, I created a second function to directly scrape the URLs from the results page of a search:

In [ ]:

getlinks<-function(url){
  linksb<-read_html(url)%>%htmlParse(asText = TRUE)%>%xmlRoot()%>%xpathSApply("//*[contains(@class,'title')]", xmlGetAttr, 'href')
  linksb[sapply(linksb, is.null)] <- NULL
  linksb<-as.character(linksb)
  linksb<-paste("https://www.indeed.co.uk", linksb,sep="")
    } 

Interestingly, as some sponsored links are always present in the results page, the total number of job offer URLs scraped is slightly superior to the expected number, which for our purpose doesn’t really create particular issues. The final step was to put everything together:

In [ ]:
scrapeindeed<-function(urlres) {
  linksbb<-getlinks(urlres)
  jobtitles<-lapply(FUN=getjobtitle, linksbb)%>%plyr::ldply(rbind)%>%mutate_if(is.factor,as.character)
  jobsdesc<-lapply (FUN=getjobdescindeed, linksbb)%>%plyr::ldply(rbind)%>%mutate_if(is.factor,as.character)
  jobmeta<-lapply (FUN=getmetadataindeed, linksbb)%>%plyr::ldply(rbind)%>%mutate_if(is.factor,as.character)
  tobemoved <- grepl("£", jobmeta[,2])#as salary can fall in the second column if location is missing, single out all the salary entries…
  jobmeta[tobemoved,3 ]<-jobmeta[tobemoved,2 ] #...to move them to the third column
  jobmeta[tobemoved,2 ] <-NA #leaving a NA on their place in the second column 
  final<-cbind(jobtitles,jobsdesc,jobmeta)%>%`colnames<-`(c("Title", "Description", "Location","Type","Salary"))
  final
}

###############
#test 
##
results1<-scrapeindeed("https://www.indeed.co.uk/jobs?as_and=&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&salary=&radius=25&l=ne18&fromage=3&limit=50&sort=&psf=advsrch")

In [ ]:
print.table(results1[1,], justify="left", width=20)

This function, fed with the URL of the results page, can extract all the data needed and store them in a dataframe of five columns. The last step was to fed the function with all the results pages relevant, which I did (in this case manually), for all the job offers published in the last three days before Saturday 18th May within 25 miles of the postcode NE18 (Newcastle Upon Tyne). The results were then merged together, the data are available <a href="https://www.dropbox.com/s/fa3hhcfi5qkqfp1/totaljobs.txt?dl=0">here</a>  in .txt format.

As you can see, there is still much to do in terms of data cleaning before starting to work, which is what we will see in the next section. 


## Part II – Data cleaning

We have now a still rather messy dataframe requiring some data cleaning before being ready to be used. Although the description part can be considered OK to be processed within STM or some other package specifically meant to work with textual data (like <a href="https://quanteda.io/">quanteda</a>), the metadata requires further massaging. Here I focus mostly on location (as I am interested in trying to perform some kind of spatial data analysis), and the salary column, as it contains quantitative information that is basically impossible to analyse in its present form. 

### Location
The location metadata is rather vague and not uniform (for most offers, the municipality is provided, but for other entries the region is mentioned instead, only a very few report the postcode), so what we can in fact do with it is rather limited. However, as this is an experiment to have fun and explore, we can try to make the most out of it nevertheless, testing what could then be achieved with better data. 
What I try here is basically to extrapolate the coordinates corresponding to the location of each offer, so that it can easily be geo-referred. This can be achieved relatively easily with <a href="https://www.r-bloggers.com/osm-nominatim-with-r-getting-locations-geo-coordinates-by-its-address/">nominatim_osm</a>, a little sweet function by Dmitry Kisler which exploit the OpenStreetMap API (there is an alternative based on google geocoding api implemented in ggmap, but it has a limit of 2500 requests per day).  The function works best with full addresses, but the municipality name is enough to get the coordinates of this (as far as it is present in OSM database). On the other hand, it doesn’t really like postcodes, so we preliminary remove them from the location column, extracting all the strings containing a digit (for convenience I have loaded the dataset prepared before, with the offers published in the last three days before Saturday 18th May within 25 miles of the postcode NE18):

In [81]:
suppressWarnings(library(jsonlite))
suppressWarnings(library(plyr))
suppressWarnings(library(sjmisc))
suppressWarnings(library(readr))
suppressWarnings(library(tidyr))
suppressWarnings(library(kulife))
suppressWarnings(library(stringr))
suppressWarnings(library(glue))
suppressWarnings(library(dplyr))
suppressWarnings(library(purrr))


totaljobs<-read.table("totaljobs.txt")# we recover the data previously scraped
totaljobs<-totaljobs[!duplicated(totaljobs),] # get rid of duplicates 
totaljobs$Location<-(str_extract(totaljobs$Location, "(\\b[^\\d]+\\b)")) # extract only the words without digits
totaljobs$Location[24]#verify

[1] "Durham"

Now we can directly fed the dataframe to the nominatim_osm function (the version I use here has only a small modification to ignore NA values, note that it might be a bit time consuming depending on speed of connection and size of the dataset):

In [82]:
nominatim_osmMod <- function(address = NULL)
{
  if(suppressWarnings(is.na(address)))
    return(data.frame())
  tryCatch(
    d <- jsonlite::fromJSON( 
      gsub('\\@addr\\@', gsub('\\s+', '\\%20', address), 
           'http://nominatim.openstreetmap.org/search/@addr@?format=json&addressdetails=0&limit=1')
    ), error = function(c) return(data.frame())
  )
  if(length(d) == 0) return(data.frame())
  return(data.frame(lon = as.numeric(d$lon), lat = as.numeric(d$lat)))
}#slightly modified to deal with NA instead of NULL



In [83]:
JobsCoord<- lapply(totaljobs$Location, nominatim_osmMod)
filter<-lapply(JobsCoord, is_empty)
JobsCoord[unlist(filter)]<-0
JobsCoorddf <- ldply(JobsCoord, data.frame)

JobsCoorddf[3]<-1:length(JobsCoorddf[,1])#ordinal index for coord
totaljobs[,6]<-(1:length(totaljobs[,1]))#ordinal index for totaljobs

names(totaljobs)[6]<-"OrdIndex"
names(JobsCoorddf)[3]<-"OrdIndex"

totaljobsCoord <- merge(totaljobs,JobsCoorddf, by="OrdIndex")


In [84]:
write.table(totaljobsCoord, "totaljobsCoord.txt",sep="\t")

The resulting dataset has the coordinates for the entity in the location column. 

In [85]:
totaljobsCoord[23,4:8]

,Location,Type,Salary,lon,lat
23,Newcastle upon Tyne,NA,"£21,414 - £22,658 a year",-1.613157,54.97385


There is a final issue to be noted, as in case of homonyms the osm API returns the most relevant result, which might not be the one relevant in our case. In the data we are using, for example, there is a “Washington” which happens to be a town in Tyne and Wear, which is unlikely to be considered the most relevant result by osm. As the entries in question are relatively few, I opted for manual intervention for most cases, an alternative workaround could be to add “, UK” at the end of each location cell, although it is a solution not devoid of side effects.

### Salary

In its present form, the salary column is basically useless for any practical purposes, as we have rates by different units (from hours to year), in some cases a single value is given, in others a range.  
The first step I took to sort this out is to extract the numeric values from the column. This can be done with grepexpr and string, which are conveniently combined in the <a href="http://stla.github.io/stlapblog/posts/Numextract.html#">Numextract function</a> by Ramnath Vaidyanathan (here I used the dataframe with the coordinates columns, but of course this procedure will work on the original dataframe as well):

In [86]:
totaljobsCoord$Salary<-as.character(totaljobsCoord$Salary)
totaljobsCoord$Salary<-gsub(",", "", totaljobsCoord$Salary)#remove commas from digits in thousands

Numextract <- function(string){
  unlist(regmatches(string,gregexpr("[[:digit:]]+\\.*[[:digit:]]*",string)))
}#curtesy of http://stla.github.io/stlapblog/posts/Numextract.html


totaljobsCoord$rate<-lapply(totaljobsCoord$Salary, Numextract)%>%plyr::ldply(rbind)


The results are stored in a dataframe within a dataframe, which we'd better store in two distinct columns for ease of use:

In [87]:
totaljobsCoord$minrate<-as.numeric(as.character(unlist(totaljobsCoord$rate[1])))#min rate
totaljobsCoord$maxrate<-as.numeric(as.character(unlist(totaljobsCoord$rate[2])))#max rate

totaljobsCoord$maxrate[!complete.cases(totaljobsCoord$maxrate)]<-totaljobsCoord$minrate[!complete.cases(totaljobsCoord$maxrate)]#in case there is no range but only a single entry, the same value is repeted in both columns

totaljobsCoordRates <- select(totaljobsCoord, -rate)#get rid of the original


The last step is to create a column extracting the factor by which the rate is computed, which can be relatively easy done with str_extract. We can preliminary check which factors to look for:

In [89]:
print(unique(gsub("\\b\\d+\\b","",totaljobsCoord$Salary))#find out the possible factors
)

 [1] "£ a year"        "£. - £. an hour" "£ an hour"       NA               
 [5] "£. an hour"      "£ - £ a year"    "£ - £ an hour"   "£ a week"       
 [9] "£ - £ a month"   "£ a day"         "£ - £ a day"     "£. - £. a day"  


In [90]:
totaljobsCoordRates$rateby<-(str_extract(totaljobsCoord$Salary, "(\\b year|day|month|week|hour\\b)"))

This makes the data more easily comparable:

In [91]:
totaljobsCoordRates[c(1:3),c(6,9,10,11)]

Salary,minrate,maxrate,rateby
£28283 a year,28283.0,28283.0,year
£8.50 - £8.70 an hour,8.5,8.7,hour
£18000 a year,18000.0,18000.0,year


As a final step, we store the resulting dataframe in a .txt for future retrieval. Depending on the type of analysis we want to perform, there is still further cleaning and massaging to be done, but for now this can be considered a workable starting point. In the next sessions I will start to use STM to do some actual analysis on the data.

In [ ]:
write.table(totaljobsCoordRates,"totaljobsCoordRates.txt",sep="\t")

## Functions
In the repo you can find the procedures seen above as functions:
- <a href="https://github.com/fracab/STMIndeed/blob/master/CoordIndeedG.R">CoordIndeedG.R</a>: extrapolate coordinates and store them in two new columns;
- <a href="https://github.com/fracab/STMIndeed/blob/master/CleanSalaryIndeedG.R">CleanSalaryIndeedG.R</a>: extrapolate min and max salary and factor by which the rate is computed;
- <a href="https://github.com/fracab/STMIndeed/blob/master/CleanIndeedG.R">CleanIndeedG.R</a>: incorporate both functions
- <a href="https://github.com/fracab/STMIndeed/blob/master/ScrapingIndeedCodeG.R">ScrapingIndeedCodeG.R</a>: the function presented in the first section to scrape data from Indeed. 